In [ ]:
from pathlib import Path as pth

root_dir = pth.cwd()

blender_path = "C:/Program Files/Blender Foundation/Blender 3.4/blender" # path to blender exe (without .exe)
blend_file_path = root_dir / "blender" / "scene.blend"
script_path = root_dir / "src" / "main.py"
output_path = root_dir / "renders"

In [ ]:
from shutil import rmtree
from src import config

config.set_seed(42)
n_samples_to_generate = 2

config_dir_path = pth("config")

# clear previous configs
if config_dir_path.is_dir():
    rmtree(config_dir_path)
config_dir_path.mkdir(parents=True)

for i in range(n_samples_to_generate):
    sample = config.Config()
    config_path = config_dir_path / f"sample_{i:08d}.json"
    config.write_config(config_path, sample)

In [ ]:
# clear output directory
if output_path.is_dir():
    rmtree(output_path)
    output_path.mkdir()

device = "OPTIX" # either CPU, CUDA or OPTIX

!"$blender_path" "$blend_file_path" --background --factory-startup --threads 0 --engine CYCLES --enable-autoexec --python "$script_path" -- --cycles-device "$device"